# asyncio를 이용한 동시성

동시성은 한 번에 많은 것을 다룬다.   
병렬성은 한 번에 많은 것을 한다.   
똑같지는 않지만, 연관성은 있다.   
동시성은 구조, 병렬성은 실행에 관한 것이다.   
동시성은 병렬화할 수 있는 문제를 해결하기 위해 해결책을 구조화하는 방법을 제공한다.

## 스레드와 코루틴 비교

http://bit.ly/1Ox3vWA

In [ ]:
# spinner_thread.py python >= 3.7
import threading
import itertools
import time


def spin(msg, done):  # <1>
    for char in itertools.cycle('|/-\\'):  # <3>
        status = char + ' ' + msg
        # \r moves the cursor to the beginning of the line and then keeps outputting characters as normal
        print(status, flush=True, end='\r') 
        if done.wait(.1):  # <5> 내부 플래그가 True가 될 때까지 blocking / timeout : 0.1s
            break
    print(' ' * len(status), end='\r')

def slow_function():  # <7>
    # pretend waiting a long time for I/O
    time.sleep(3)  # <8>
    return 42


def supervisor():  # <9>
    done = threading.Event() # thread 간의 간단한 통신을 위해 사용되는 객체
    spinner = threading.Thread(target=spin,
                               args=('thinking!', done))
    print('spinner object:', spinner)  # <10>
    spinner.start()  # <11>
    result = slow_function()  # <12>
    done.set()  # <13> 내부 플래그를 True로 설정
    spinner.join()  # <14>
    return result


def main():
    result = supervisor()  # <15>
    print('Answer:', result)

'''
Event 객체

thread 간의 간단한 통신을 위해 사용되는 객체
is_set() : 내부 플래그가 True면 그때만 True 반환
set() : 내부 플래그를 True로 설정
clear() : 내부 플래그를 False로 설정
wait(timeout=None) : 내부 플래그가 True가 될때까지 blocking함. 
'''


In [ ]:
main()

In [11]:
import asyncio
import itertools


async def spin(msg):  # <1>
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        try:
            await asyncio.sleep(.1)  # <2>
        except asyncio.CancelledError:  # <3>
            break
    print(' ' * len(status), end='\r')


async def slow_function():  # <4>
    # pretend waiting a long time for I/O
    await asyncio.sleep(3)  # <5>
    return 42


async def supervisor():  # <6>
    spinner = asyncio.create_task(spin('thinking!'))  # <7>
    print('spinner object:', spinner)  # <8>
    result = await slow_function()  # <9>
    spinner.cancel()  # <10>
    return result


def main():
    result = asyncio.run(supervisor())  # <11>
    print('Answer:', result)


In [12]:
main()

RuntimeError: asyncio.run() cannot be called from a running event loop

## asyncio와 aiohttp로 내려받기

In [1]:
import os
import time
import sys
import asyncio  # <1>

import aiohttp  # <2>


POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


async def get_flag(session, cc):  # <3>
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    async with session.get(url) as resp:        # <4>
        return await resp.read()  # <5>


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


async def download_one(session, cc):  # <6>
    image = await get_flag(session, cc)  # <7>
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


async def download_many(cc_list):
    async with aiohttp.ClientSession() as session:  # <8>
        res = await asyncio.gather(                 # <9>
            *[asyncio.create_task(download_one(session, cc))
                for cc in sorted(cc_list)])

    return len(res)


def main():  # <10>
    t0 = time.time()
    count = asyncio.run(download_many(POP20_CC))
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))



In [2]:
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
main()

RuntimeError: asyncio.run() cannot be called from a running event loop

## 블로킹 호출을 에둘러 실행하기